In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd /content/drive/My Drive/CS 577 Project/helper functions
%ls

/content/drive/My Drive/CS 577 Project/helper functions
data_loader.py  models.py  __pycache__/  utility.py  vgg_nets.py  video_utility.py


In [ ]:
pip install -U scikit-image

In [12]:
import cv2 as cv
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim


import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from models import U2NET # full size version 173.6 MB
from models import U2NETP # small version u2net 4.7 MB


PATH = '/content/drive/My Drive/CS 577 Project'

%cd /content/drive/My Drive/CS 577 Project/
%ls

/content/drive/My Drive/CS 577 Project
'Code Reference.txt'    'helper functions'/   PhotoWakeUp/    StyleTransfer.ipynb   U2Net.ipynb
'CS 577 Project.ipynb'   images/              __pycache__/    StyleTransfer.py      U2Net.py
 Grabcut.ipynb           output_experiment/   saved_models/   test_data/


In [13]:
# normalize the predicted SOD probability map


def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

def save_output(image_name,pred,d_dir):

    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    img_name = image_name.split(os.sep)[-1]
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

    pb_np = np.array(imo)

    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]

    imo.save(d_dir+imidx+'.png')


def image_separation(figure_name):


    mask_img_path = PATH + '/test_data/images/u2net_results/' + figure_name + '.png'
    og_img_path = PATH + '/test_data/images/content/' + figure_name + '.jpg'

    save_out_path = PATH + '/test_data/images/output/foreground/' + figure_name + '_fg.png'
    save_bckgr_path = PATH + '/test_data/images/output/background/'+ figure_name + '_bg.png'

    #mask image
    subimage=Image.open(mask_img_path)
    #original image
    original=Image.open(og_img_path)
    #background image
    bckgrimage=Image.open(og_img_path)


    subimage=subimage.convert("RGBA")
    original=original.convert("RGBA")
    bckgrimage=original.convert("RGBA")

    subdata=subimage.getdata()
    ogdata=original.getdata()
    bckdata=bckgrimage.getdata()

    newdata=[]
    newdata_bckg=[]


    for i in range(subdata.size[0]*subdata.size[1]):
      if subdata[i][0]==0 and subdata[i][1]==0 and subdata[i][2]==0:
        newdata.append((255,255,255,0))
        newdata_bckg.append(bckdata[i])
      else:
        newdata.append(ogdata[i])
        newdata_bckg.append((255,255,255,0))

    subimage.putdata(newdata)
    subimage.save(save_out_path,"PNG")


    bckgrimage.putdata(newdata_bckg)
    bckgrimage.save(save_bckgr_path,"PNG")

    return subimage, bckgrimage

def compile_result(image_name,pred):
    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

#img_result = compile_result(img_name_list[i_test],pred)
#def compute_mask(figure_name, model_name, image_dir, prediction_dir, model_dir, img_name_list):


def main():
    # # --------- 1. get image path and name ---------
    model_name='u2net'#'u2netp'

    image_dir=os.path.join(os.getcwd(),'test_data','images','content')
    prediction_dir=os.path.join(os.getcwd(),'test_data','images',model_name+'_results'+os.sep)
    model_dir = os.path.join(os.getcwd(),'saved_models',model_name,model_name+'.pth')

    img_name_list = glob.glob(image_dir + os.sep + '*')

    # --------- 2. dataloader ---------
    #1. dataloader

    test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                        lbl_name_list = [],
                                        transform=transforms.Compose([RescaleT(320),
                                                                      ToTensorLab(flag=0)]) )
    test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                        batch_size=1,
                                        shuffle=False,
                                        num_workers=1)

    # --------- 3. model define ---------
    if(model_name=='u2net'):
        print("...load U2NET---173.6 MB")
        net = U2NET(3,1)
    elif(model_name=='u2netp'):
        print("...load U2NEP---4.7 MB")
        net = U2NETP(3,1)
    net.load_state_dict(torch.load(model_dir,map_location=torch.device('cpu')))
    if torch.cuda.is_available():
        net.cuda()
    net.eval()



    # --------- 4. inference for each image ---------
    for i_test, data_test in enumerate(test_salobj_dataloader):

        print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

        inputs_test = data_test['image']
        inputs_test = inputs_test.type(torch.FloatTensor)

        if torch.cuda.is_available():
            inputs_test = Variable(inputs_test.cuda())
        else:
            inputs_test = Variable(inputs_test)

        d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

        # normalization
        pred = d1[:,0,:,:]
        pred = normPRED(pred)

        # save results to test_results folder
        if not os.path.exists(prediction_dir):
            os.makedirs(prediction_dir, exist_ok=True)
        save_output(img_name_list[i_test],pred,prediction_dir)

        del d1,d2,d3,d4,d5,d6,d7


if __name__ == "__main__":
     main()

...load U2NET---173.6 MB
inferencing: 5.jpg
inferencing: figures.jpg
inferencing: parrot.jpg
inferencing: stylized_parrot.jpg


In [14]:
#Specify which image we want to work on.

figure_name = 'stylized_parrot'

In [15]:
image_separation(figure_name)

(<PIL.Image.Image image mode=RGBA size=769x512>,
 <PIL.Image.Image image mode=RGBA size=769x512>)